In [1]:
!git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null
from seminaire.evaluation.jour4.textclassification.project import 

SyntaxError: invalid syntax (4111019778.py, line 3)

### Évaluation d'un modèle de classification

### Agenda
* Quel est l'objectif de l'évaluation d'un modèle, et quelles sont les procédures d'évaluation les plus courantes ?
* Quel est l'usage de la précision de classification, et quelles sont ses limites ?
* Comment une matrice de confusion décrit-elle la performance d'un classificateur ?
* Quelles métriques peuvent être calculées à partir d'une matrice de confusion ?
* Comment pouvez-vous ajuster la performance d'un classificateur en changeant le seuil de classification ?
* Quel est le but d'une courbe ROC ?
* Comment l'aire sous la courbe (AUC) diffère-t-elle de la précision de la classification ?

### Examen de l'évaluation des modèles
* Besoin d'un moyen de choisir entre les modèles : différents types de modèles, paramètres de réglage et caractéristiques.
* Utiliser une procédure d'évaluation de modèle pour estimer la capacité d'un modèle à généraliser les données hors échantillon.
* Nécessite une métrique d'évaluation de modèle pour quantifier la performance du modèle.

### Procédures d'évaluation de modèles
1. Formation et test sur les mêmes données
    * Récompense les modèles trop complexes qui "surajustent" les données de formation et ne généralisent pas nécessairement.
2. Division Train/test
    * Divise l'ensemble de données en deux parties, de sorte que le modèle peut être formé et testé sur des données différentes.
    * Meilleure estimation de la performance hors échantillon, mais toujours une estimation à "haute variance".
    * Utile en raison de sa rapidité, de sa simplicité et de sa flexibilité.
3. Validation croisée K-fold
    * Créez systématiquement "K" séparations train/test et faites la moyenne des résultats ensemble.
    * Meilleure estimation des performances hors échantillon.
    * Fonctionne "K" fois plus lentement que la division train/test.

### Métriques d'évaluation du modèle
* Problèmes de régression : Erreur absolue moyenne, erreur quadratique moyenne, erreur quadratique moyenne.
* Problèmes de classification : Précision de la classification



### Précision de la classification

Jeu de données sur le diabète des Indiens Pima provenant du dépôt d'apprentissage automatique de l'UCI.

In [ ]:
# read the data into a Pandas DataFrame
import pandas as pd
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data'
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
pima = pd.read_csv(url, header=None, names=col_names)

In [ ]:
# print the first 5 rows of data
pima.head()

Question : Peut-on prédire le statut diabétique d'un patient à partir de ses mesures de santé ?

In [ ]:
# define X and y
feature_cols = ['pregnant', 'insulin', 'bmi', 'age']
X = pima[feature_cols]
y = pima.label

In [ ]:
# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
# make class predictions for the testing set
y_pred_class = logreg.predict(X_test)

Précision de la classification : pourcentage de prédictions correctes

In [ ]:
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

Précision nulle : précision qui pourrait être obtenue en prédisant toujours la classe la plus fréquente.

In [ ]:
# examine the class distribution of the testing set (using a Pandas Series method)
y_test.value_counts()

In [ ]:
# calculate the percentage of ones
y_test.mean()

In [ ]:
# calculate the percentage of zeros
1 - y_test.mean()

In [ ]:
# calculate null accuracy (for binary classification problems coded as 0/1)
max(y_test.mean(), 1 - y_test.mean())

In [ ]:
# calculate null accuracy (for multi-class classification problems)
y_test.value_counts().head(1) / len(y_test)

Comparaison des valeurs de réponse réelles et prédites

In [ ]:
# print the first 25 true and predicted responses
from __future__ import print_function
print('True:', y_test.values[0:25])
print('Pred:', y_pred_class[0:25])

Conclusion :

* La précision de la classification est la mesure de classification la plus facile à comprendre.
* Mais, elle ne vous indique pas la distribution sous-jacente des valeurs de réponse.
* Et elle ne vous dit pas quels "types" d'erreurs votre classificateur commet.

### Matrice de confusion (confusion matrix)

Tableau qui décrit la performance d'un modèle de classification

In [ ]:
# IMPORTANT: first argument is true values, second argument is predicted values
print(metrics.confusion_matrix(y_test, y_pred_class))

Chaque observation de l'ensemble de test est représentée dans une seule case.
Il s'agit d'une matrice 2x2 car il y a deux classes de réponses.
Le format présenté ici n'est pas universel

Terminologie de base

* Vrais positifs (TP) : nous avons correctement prédit qu'ils ont le diabète.
* Vrais négatifs (TN) : nous avons correctement prédit qu'ils ne sont pas diabétiques.
* Faux positifs (FP) : nous avons prédit à tort qu'ils sont diabétiques (une "erreur de type I").
* Faux négatifs (FN) : nous avons prédit à tort qu'ils ne sont pas diabétiques (une "erreur de type II").

In [ ]:
# print the first 25 true and predicted responses
print('True:', y_test.values[0:25])
print('Pred:', y_pred_class[0:25])

In [ ]:
# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred_class)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

### Métriques calculées à partir d'une matrice de confusion

Précision de la classification : Globalement, combien de fois le classificateur est-il correct ?

In [ ]:
print((TP + TN) / float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_test, y_pred_class))

Erreur de classification : Globalement, combien de fois le classificateur est-il incorrect ?

Également connu sous le nom de "taux de classification erronée".

In [ ]:
print((FP + FN) / float(TP + TN + FP + FN))
print(1 - metrics.accuracy_score(y_test, y_pred_class))

Sensibilité : Lorsque la valeur réelle est positive, combien de fois la prédiction est-elle correcte ?

Dans quelle mesure le classificateur est-il "sensible" à la détection d'instances positives ?
Également connu sous le nom de "taux de vrais positifs" ou "rappel".

In [ ]:
print(TP / float(TP + FN))
print(metrics.recall_score(y_test, y_pred_class))

Spécificité : Lorsque la valeur réelle est négative, combien de fois la prédiction est-elle correcte ?

Dans quelle mesure le classificateur est-il "spécifique" (ou "sélectif") pour prédire des instances positives ?

In [ ]:
print(TN / float(TN + FP))

Taux de faux positifs : Lorsque la valeur réelle est négative, combien de fois la prédiction est-elle incorrecte ?

In [ ]:
print(FP / float(TN + FP))

La précision : Lorsqu'une valeur positive est prédite, combien de fois la prédiction est-elle correcte ?

Quelle est la "précision" du classifieur lorsqu'il prédit des instances positives ?

In [ ]:
print(TP / float(TP + FP))
print(metrics.precision_score(y_test, y_pred_class))

De nombreuses autres métriques peuvent être calculées : le score F1, le coefficient de corrélation de Matthews, etc.

Conclusion :

La matrice de confusion vous donne une image plus complète de la performance de votre classificateur.
Elle vous permet également de calculer diverses métriques de classification, et ces métriques peuvent guider votre sélection de modèle.
Sur quelles métriques devez-vous vous concentrer ?

Le choix de la métrique dépend de votre objectif commercial
Filtre anti-spam (la classe positive est "spam") : Optimisez la précision ou la spécificité car les faux négatifs (les spams vont dans la boîte de réception) sont plus acceptables que les faux positifs (les non-spams sont détectés par le filtre anti-spam).
Détecteur de transactions frauduleuses (la classe positive est "fraude") : Optimisez la sensibilité parce que les faux positifs (les transactions normales qui sont signalées comme pouvant être frauduleuses) sont plus acceptables que les faux négatifs (les transactions frauduleuses qui ne sont pas détectées).

1. Jour 1
    * Variables
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.1%20helloworld.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.2%20%C3%A9viter%20les%20errreurs%20de%20nommage.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.3%20mini-project.ipynb#scrollTo=RPB2xCMdA6lV)
    * Strings
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.1%20concat.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.2%20string_methods.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.3%20mini-project.ipynb)
    * Opérations
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.1%20math.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.2%20bool%C3%A9en.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.3%20mini-project.ipynb)

2. Jour 2
    * Listes
        * [Définition](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.1%20d%C3%A9finition%20liste.ipynb?hl=fr)
        * [Strings et listes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.2%20String%20as%20list%20of%20characters.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.3%20mini-project.ipynb?hl=fr)
    * Fonctions
        * [Définition I](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.1%20d%C3%A9finition%20fonctions.ipynb?hl=fr)
        * [Définition II](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.2%20scope%20et%20fonctions%20imbriqu%C3%A9es.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.3%20mini-project.ipynb?hl=fr) 
    * Librairies
        * [Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.1%20Pandas.ipynb?hl=fr)
        * [Matplotlib](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.2%20Matplotlib.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.3%20mini-project.ipynb?hl=fr) 
3. Jour 3
    * Introduction à la NLP
        * [Charger des un corpus](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.1%20Accessing%20Text.ipynb?hl=fr)
        * [Traitement de texte dans Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.2%20Working%20with%20text%20data%20in%20pandas.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.3%20mini-project.ipynb?hl=fr)
    * Segmentation
        * [Segmentation de tokens](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.1%20Token%20segmentation.ipynb?hl=fr)
        * [Segmentation de phrase](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.2%20Sentence%20segmentation.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.3%20mini-project.ipynb?hl=fr)
    * Nettoyage de texte
        * [Stopwords](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.1%20stopwords.ipynb?hl=fr)
        * [Normalisation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.2%20Normalizing%20Text.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.3%20mini-project.ipynb?hl=fr)
4. Jour 4
    * Apprentissage supervisé
        * [Régression linéaire](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.1%20linear%20regression.ipynb?hl=fr)
        * [Evaluation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.2%20evaluale.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.3%20mini-project.ipynb?hl=fr)
    * Pré-traitement de texte
        * [Featurization de textes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.1%20text%20featurization.ipynb?hl=fr)
        * [Featurization de labels](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.2%20label%20featurization.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.3%20mini-project.ipynb?hl=fr)
    * Classification de texte
        * [EDA](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Apprentissage supervisé textuel](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.3%20mini-project.ipynb?hl=fr)
5. Jour 5
    * [Projet final](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour5/final-project.ipynb?hl=fr)